# Violin plot

In [1]:
# Imports
import numpy as np
import pandas as pd
import os

# figure imports
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots


# Import data base
main_dir = '/Users/uriel/disks/meso_shared'
project_dir = 'RetinoMaps'
subjects = ['sub-02']
subject = 'sub-02'
tsv_dir ='{}/{}/derivatives/pp_data/{}/fsnative/prf/tsv'.format(main_dir, 
                                                                project_dir, 
                                                                subject)

In [2]:
# Load data
data = pd.read_table('{}/{}_task-prf_loo.tsv'.format(tsv_dir,subject))


In [3]:
# Filter data
ecc_th = [0, 15]
size_th= [0.1, 20]
rsq_th = [0.05, 1]

# Replace all data outer threshold with NaN data
# Replace all data outer threshold with NaN data
data.loc[(data.prf_ecc < ecc_th[0]) | (data.prf_ecc > ecc_th[1]) | 
         (data.prf_size < size_th[0]) | (data.prf_size > size_th[1]) | 
         (data.prf_loo_r2 <=rsq_th[0])] = np.nan

data = data.dropna()

rois = pd.unique(data.rois)
mask = pd.notnull(data.subject)
subjects = pd.unique(data.subject[mask])

In [4]:
# Define colors
roi_colors = px.colors.sequential.Sunset[:4] + px.colors.sequential.Rainbow[:]

In [5]:
roi = 'V1'
df = data.loc[(data.subject == subject) & (data.rois == roi)]

print(df.prf_size.mean())

1.4616783095108068


## Make Violin plot comparison R2 / CF-projected Size / CF-projected Eccentricity

In [15]:
fig_dir = '/Users/uriel/Downloads/'

In [16]:
rows, cols = 2,2
fig_height, fig_width = 1080,1920

for i, subject in enumerate(subjects):
    fig = make_subplots(rows=rows, cols=cols, 
                        print_grid=False, 
                        vertical_spacing=0.08, 
                        horizontal_spacing=0.1)
    
    for j, roi in enumerate(rois):
        
        df = data.loc[(data.subject == subject) & (data.rois == roi)]
        
        # df = df.sort_values('prf_rsq_loo', ascending=False)
        # df = df.head(250)

        # pRF loo r2
        fig.add_trace(go.Violin(x=df.rois[df.rois==roi], 
                                y=df.prf_loo_r2, 
                                name=roi, 
                                showlegend=True, 
                                legendgroup='loo', 
                                points=False, 
                                scalemode='width', 
                                width=0.75, 
                                side='negative', 
                                line_color = roi_colors[j], 
                                meanline_visible=True), 
                      row=1, col=1)
        
        
        # pRF r2
        fig.add_trace(go.Violin(x=df.rois[df.rois==roi], 
                                y=df.prf_rsq, 
                                name=roi, 
                                showlegend=False, 
                                legendgroup='avg', 
                                points=False, 
                                scalemode='width', 
                                width=0.75, 
                                side='positive', 
                                line_color = roi_colors[j], 
                                meanline_visible=True, 
                                fillcolor='rgb(255,255,255)'), 
                      row=1, col=1)
        
        
        # pRF size
        fig.add_trace(go.Violin(x=df.rois[df.rois==roi], 
                                y=df.prf_size, 
                                name=roi, 
                                showlegend=False, 
                                legendgroup='avg', 
                                points=False, 
                                scalemode='width', 
                                width=0.75, 
                                line_color = roi_colors[j], 
                                meanline_visible=True), 
                      row=1, col=2)
        
        # pRF ecc
        fig.add_trace(go.Violin(x=df.rois[df.rois==roi], 
                                y=df.prf_ecc, 
                                name=roi, 
                                showlegend=False, 
                                legendgroup='avg', 
                                points=False, 
                                scalemode='width', 
                                width=0.75,  
                                line_color = roi_colors[j], 
                                meanline_visible=True), 
                      row=2, col=1)
 
        
        # Set axis titles only for the left-most column and bottom-most row
        fig.update_yaxes(range=[0,1], 
                         title_text='r2', 
                         row=1, col=1)
        
        fig.update_yaxes(range=[0,20], 
                         title_text='pRF size (dva)', 
                         row=1, col=2)
        
        fig.update_yaxes(range=[0,15], 
                         nticks=4, 
                         title_text='pRF eccentricity (dva)', 
                         row=2, col=1)
        
        fig.update_xaxes(showline=True, 
                         ticklen=0, 
                         linecolor=('rgba(255,255,255,0)'), 
                         tickfont=dict(size=18))
        
        fig.update_traces(spanmode='manual', 
                          span=[0,1], 
                          row=1, col=1)  
        
        fig.update_traces(spanmode='manual', 
                          span=[0.1,20], 
                          row=1, col=2)
        
        fig.update_traces(spanmode='manual', 
                          span=[0,15], 
                          row=2, col=1)
        
        
    fig.update_layout(height=fig_height, 
                      width=fig_width, 
                      showlegend=True, 
                      legend=dict(orientation="h", 
                                  yanchor='top', 
                                  y=1.15, 
                                  xanchor='left', 
                                  x=0.22, 
                                  traceorder='normal', 
                                  itemwidth=50), 
                      template='simple_white', 
                      font=dict(size=16))
    #fig.show()
    fig.write_image("{}/{}_prf_cf_rsq_size_ecc.pdf".format(fig_dir, subject)) 
    